In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}



E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130963 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import sys
import numpy as np
import nltk
nltk.download('punkt')

from sklearn.ensemble import GradientBoostingClassifier
from drive.FNC_challenge.feature_engineering import refuting_features, polarity_features, hand_features, gen_or_load_feats
from drive.FNC_challenge.feature_engineering import word_overlap_features
from drive.FNC_challenge.utils.dataset import DataSet
from drive.FNC_challenge.utils.generate_test_splits import kfold_split, get_stances_for_folds
from drive.FNC_challenge.utils.score import report_score, LABELS, score_submission
from keras.layers import Embedding, Input
from keras.layers.recurrent import LSTM
from keras.layers.merge import concatenate
from keras.layers.core import Dense
from keras.models import Model, load_model
from drive.FNC_challenge.utils.system import parse_params, check_version
from keras import optimizers

from keras.callbacks import Callback

from drive.FNC_challenge.utils.score import LABELS, score_submission


def append_to_loss_monitor_file(text, filepath):
    with open(filepath, 'a+') as the_file:
        the_file.write(text+"\n")

class EarlyStoppingOnF1(Callback):
    """
    Prints some metrics after each epoch in order to observe overfitting
                https://github.com/fchollet/keras/issues/5794
                custom metrics: https://github.com/fchollet/keras/issues/2607
    """

    def __init__(self, epochs,
                 X_test_claims,
                 X_test_orig_docs,
                 y_test, loss_filename, epsilon=0.0, min_epoch=15, X_test_nt=None):
        self.epochs = epochs
        self.patience = 2
        self.counter = 0
        self.prev_score = 0
        self.epsilon = epsilon
        self.loss_filename = loss_filename
        self.min_epoch = min_epoch
        self.X_test_nt = X_test_nt
        # self.print_train_f1 = print_train_f1

        # self.X_train_claims = X_train_claims
        # self.X_train_orig_docs = X_train_orig_docs
        # self.X_train_evid = X_train_evid
        # self.y_train = y_train

        self.X_test_claims = X_test_claims
        self.X_test_orig_docs = X_test_orig_docs
        self.y_test = y_test
        Callback.__init__(self)

    def on_epoch_end(self, epoch, logs={}):
        if epoch + 1 < self.epochs:
            from sklearn.metrics import f1_score

            # get prediction and convert into list
            if type(self.X_test_orig_docs).__module__ == np.__name__ and type(self.X_test_nt).__module__ == np.__name__:
                predicted_one_hot = self.model.predict([
                    self.X_test_claims,
                    self.X_test_orig_docs,
                    self.X_test_nt
                ])
            elif type(self.X_test_orig_docs).__module__ == np.__name__:
                predicted_one_hot = self.model.predict([
                    self.X_test_claims,
                    self.X_test_orig_docs,
                ])
            else:
                predicted_one_hot = self.model.predict(self.X_test_claims)
            predict = np.argmax(predicted_one_hot, axis=-1)

            """
            predicted_one_hot_train = self.model.predict([self.X_train_claims, self.X_train_orig_docs, self.X_train_evid])
            predict_train = np.argmax(predicted_one_hot_train, axis=-1)


            # f1 for train data
            f1_macro_train = ""
            if self.print_train_f1 == True:
                f1_0_train = f1_score(self.y_train, predict_train, labels=[0], average=None)
                f1_1_train = f1_score(self.y_train, predict_train, labels=[1], average=None)
                f1_macro_train = (f1_0_train[0] + f1_1_train[0]) / 2
                print(" - train_f1_(macro): " + str(f1_macro_train))"""

            predicted = [LABELS[int(a)] for a in predict]
            actual = [LABELS[int(a)] for a in self.y_test]
            # calc FNC score
            fold_score, _ = score_submission(actual, predicted)
            max_fold_score, _ = score_submission(actual, actual)
            fnc_score = fold_score / max_fold_score
            print(" - fnc_score: " + str(fnc_score))

            # f1 for test data
            f1_0 = f1_score(self.y_test, predict, labels=[0], average=None)
            f1_1 = f1_score(self.y_test, predict, labels=[1], average=None)
            f1_2 = f1_score(self.y_test, predict, labels=[2], average=None)
            f1_3 = f1_score(self.y_test, predict, labels=[3], average=None)
            f1_macro = (f1_0[0] + f1_1[0] + f1_2[0] + f1_3[0]) / 4
            print(" - val_f1_(macro): " + str(f1_macro))
            print("\n")

            header = ""
            values = ""
            for key, value in logs.items():
                header = header + key + ";"
                values = values + str(value) + ";"
            if epoch == 0:
                values = "\n" + header + "val_f1_macro;" + "fnc_score;" + "\n" + values + str(f1_macro) + str(
                    fnc_score) + ";"
            else:
                values += str(f1_macro) + ";" + str(fnc_score) + ";"
            append_to_loss_monitor_file(values, self.loss_filename)

            if epoch >= self.min_epoch - 1:  # 9
                if f1_macro + self.epsilon <= self.prev_score:
                    self.counter += 1
                else:
                    self.counter = 0
                if self.counter >= 2:
                    self.model.stop_training = True
            # print("Counter at " + str(self.counter))
            self.prev_score = f1_macro
            # print("\n")

def calculate_class_weight(y_train, no_classes=2):
    # https://datascience.stackexchange.com/questions/13490/how-to-set-class-weights-for-imbalanced-classes-in-keras
    from sklearn.utils import class_weight

    class_weight_list = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
    class_weights = {}
    for i in range(no_classes):
        class_weights[i] = class_weight_list[i]
    print(class_weights)
    return class_weights

def convert_data_to_one_hot(y_train):
    # y_test_temp = np.zeros((y_test.size, y_test.max() + 1), dtype=np.int)
    # y_test_temp[np.arange(y_test.size), y_test] = 1

    # Other option:
    #   y_train is a tensor then because of one_hot, but feed_dict only accepts numpy arrays => replace y_train with sess.run(y_train)
    #   http://stackoverflow.com/questions/34410654/tensorflow-valueerror-setting-an-array-element-with-a-sequence
    # return tf.one_hot(y_train, 4), tf.one_hot(y_test, 4)
    y_train_temp = np.zeros((y_train.size, y_train.max() + 1), dtype=np.int)
    y_train_temp[np.arange(y_train.size), y_train] = 1

    return y_train_temp

def split_X(X_train, MAX_SEQ_LENGTH_HEADS):
    # split to get [heads, docs]
    X_train_splits = np.hsplit(X_train, np.array([MAX_SEQ_LENGTH_HEADS]))
    X_train_head = X_train_splits[0]
    X_train_doc = X_train_splits[1]

    print("X_train_head.shape = " + str(np.array(X_train_head).shape))
    print("X_train_doc.shape = " + str(np.array(X_train_doc).shape))

    return X_train_head,X_train_doc

def generate_features(stances,dataset,name):
    h, b, y = [],[],[]

    for stance in stances:
        y.append(LABELS.index(stance['Stance']))
        h.append(stance['Headline'])
        b.append(dataset.articles[stance['Body ID']])

    X_overlap = gen_or_load_feats(word_overlap_features, h, b, "drive/FNC_challenge/features/overlap."+name+".npy")
    X_refuting = gen_or_load_feats(refuting_features, h, b, "drive/FNC_challenge/features/refuting."+name+".npy")
    X_polarity = gen_or_load_feats(polarity_features, h, b, "drive/FNC_challenge/features/polarity."+name+".npy")
    X_hand = gen_or_load_feats(hand_features, h, b, "drive/FNC_challenge/features/hand."+name+".npy")

    X = np.c_[X_hand, X_polarity, X_refuting, X_overlap]
    return X,y

if __name__ == "__main__":

    batch_size = 500
    epochs = 100
    word_index = 100
    EMBEDDING_DIM = 50
    MAX_LENGTH = 16
    LSTM_implementation = 2
    embeddings_index = {}
    from gensim.models import KeyedVectors
    #
    glove_file = 'drive/FNC_challenge/glove.twitter.27B.50d.txt'
    tmp_file = 'drive/FNC_challenge/word2vec.txt'

    vecmodel = KeyedVectors.load_word2vec_format(tmp_file)
    word2idx = {'_PAD': 0}

    vocab_list = [(k, vecmodel.wv[k]) for k, v in vecmodel.wv.vocab.items()]
    embeddings_matrix = np.zeros((len(vecmodel.wv.vocab.items()) + 1, vecmodel.vector_size))
    for i in range(len(vocab_list)):
        word = vocab_list[i][0]
        word2idx[word] = i + 1
        embeddings_matrix[i + 1] = vocab_list[i][1]
    print(embeddings_matrix.shape)

    # embedding_layer = Embedding(len(vecmodel.wv.vocab.items()) + 1,
    #                             EMBEDDING_DIM,
    #                             weights=[embeddings_matrix],
    #                             trainable=False,
    #                             input_length=4
    #                             )
    # parse_params()
    # competition_dataset = DataSet("competition_test")
    # X_competition, y_competition = generate_features(competition_dataset.stances, competition_dataset, "competition")
    d = DataSet("train","drive/FNC_challenge/fnc-1")
    folds, hold_out = kfold_split(d, n_folds=10, base_dir="drive/FNC_challenge/splits")
    fold_stances, hold_out_stances = get_stances_for_folds(d, folds, hold_out)

    # Load the competition dataset
    competition_dataset = DataSet("competition_test","drive/FNC_challenge/fnc-1")
    X_competition, y_competition = generate_features(competition_dataset.stances, competition_dataset, "competition")
    X_competition_LSTM, X_competition_MLP = split_X(X_competition, MAX_LENGTH)
    y_competition_one_hot = convert_data_to_one_hot(np.array(y_competition))
    Xs = dict()
    ys = dict()

    # Load/Precompute all features now
    X_holdout, y_holdout = generate_features(hold_out_stances, d, "holdout")
    X_holdout_LSTM, X_holdout_MLP = split_X(X_holdout, MAX_LENGTH)
    y_holdout_one_hot = convert_data_to_one_hot(np.array(y_holdout))
    for fold in fold_stances:
        Xs[fold], ys[fold] = generate_features(fold_stances[fold], d, str(fold))

    best_score = 0
    best_fold = None

    for fold in fold_stances:
        ids = list(range(len(folds)))
        del ids[fold]

        X_train = np.vstack(tuple([Xs[i] for i in ids]))
        y_train = np.hstack(tuple([ys[i] for i in ids]))

        X_test = Xs[fold]
        y_test = ys[fold]
        y_train_one_hot = convert_data_to_one_hot(y_train)
        y_test_one_hot = convert_data_to_one_hot(np.array(y_test))
        X_train_LSTM, X_train_MLP = split_X(X_train, MAX_LENGTH)
        X_test_LSTM, X_test_MLP = split_X(X_test, MAX_LENGTH)
        class_weights = calculate_class_weight(y_train, no_classes=4)
        lstm_input = Input(shape=(MAX_LENGTH,), dtype='int32', name='lstm_input')
        embedding = Embedding(input_dim=len(vecmodel.wv.vocab.items())+1,  # lookup table size
                              output_dim=EMBEDDING_DIM,  # output dim for each number in a sequence
                              weights=[embeddings_matrix],
                              input_length=MAX_LENGTH,  # receive sequences of MAX_SEQ_LENGTH_CLAIMS integers
                              mask_zero=True,
                              trainable=True)(lstm_input)

        data_LSTM = LSTM(
            100, return_sequences=True, stateful=False, dropout=0.2,
            batch_input_shape=(batch_size, MAX_LENGTH, EMBEDDING_DIM),
            input_shape=(MAX_LENGTH, EMBEDDING_DIM), implementation=LSTM_implementation
        )(embedding)
        data_LSTM = LSTM(
            100, return_sequences=False, stateful=False, dropout=0.2,
            batch_input_shape=(batch_size, MAX_LENGTH, EMBEDDING_DIM),
            input_shape=(MAX_LENGTH, EMBEDDING_DIM), implementation=LSTM_implementation
        )(data_LSTM)

        mlp_input = Input(shape=(len(X_train_MLP[0]),), dtype='float32', name='mlp_input')

        merged = concatenate([data_LSTM, mlp_input])

        dense_mid = Dense(600, kernel_regularizer=None, kernel_initializer='glorot_uniform',
                          activity_regularizer= None, activation='relu')(merged)
        dense_mid = Dense(600, kernel_regularizer=None, kernel_initializer='glorot_uniform',
                          activity_regularizer=None, activation='relu')(dense_mid)
        dense_mid = Dense(600, kernel_regularizer=None, kernel_initializer='glorot_uniform',
                          activity_regularizer=None, activation='relu')(dense_mid)
        dense_out = Dense(4, activation='softmax', name='dense_out')(dense_mid)

        model = Model(inputs=[lstm_input, mlp_input], outputs=[dense_out])

        model.compile(optimizer=optimizers.Adam(), loss='kullback_leibler_divergence',  # categorial_crossentropy
                           metrics=['accuracy']
                           )

        print("Starting training ")
        print("xtrain shape:" + str(np.array(X_train).shape))
        print("ytrain shape:" + str(np.array(y_train).shape))
        model.fit([X_train_LSTM, X_train_MLP],
                       y_train_one_hot,
                       validation_data=([X_test_LSTM, X_test_MLP], y_test_one_hot),
                       batch_size=batch_size, epochs=epochs, verbose=1,
                       callbacks=[
                           EarlyStoppingOnF1(epochs,
                                             X_test_LSTM, X_test_MLP, y_test,
                                             'model_loss', epsilon=0.0, min_epoch=10),
                       ]
        )
        print("Training finished \n")
        X_test_LSTM, X_test_MLP = split_X(X_test, MAX_LENGTH)

        predicted_one_hot = model.predict(x=[X_test_LSTM, X_test_MLP])
        predicted = np.argmax(predicted_one_hot, axis=-1)
        y_test = np.argmax(y_test_one_hot.tolist(), axis=-1)

        predicted = [LABELS[int(a)] for a in predicted]
        actual = [LABELS[int(a)] for a in y_test]
        fold_score, _ = score_submission(actual, predicted)
        max_fold_score, _ = score_submission(actual, actual)
        score = fold_score / max_fold_score

        print("Score for fold " + str(fold) + " was - " + str(score))
        if score > best_score:
            best_score = score
            best_fold = model
     
    predicted_one_hot = model.predict(x=[X_holdout_LSTM, X_holdout_MLP])
    predicted = np.argmax(predicted_one_hot, axis=-1)
    y_holdout = np.argmax(y_holdout_one_hot.tolist(), axis=-1)       
    predicted = [LABELS[int(a)] for a in predicted]
    actual = [LABELS[int(a)] for a in y_holdout]
    print("Scores on the dev set")
    report_score(actual, predicted)
    print("")
    print("")
    
    # Run on competition dataset
    predicted_one_hot_competition = model.predict(x=[X_competition_LSTM, X_competition_MLP])
    predicted = np.argmax(predicted_one_hot_competition, axis=-1)
    y_competition = np.argmax(y_competition_one_hot.tolist(), axis=-1)  
    predicted = [LABELS[int(a)] for a in predicted]
    actual = [LABELS[int(a)] for a in y_competition]

    print("Scores on the test set")
    report_score(actual, predicted)
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:201: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:202: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


(1193515, 50)
Reading dataset
Total stances: 49972
Total bodies: 1683
Reading dataset
Total stances: 25413
Total bodies: 904
X_train_head.shape = (25413, 16)
X_train_doc.shape = (25413, 28)
X_train_head.shape = (9622, 16)
X_train_doc.shape = (9622, 28)


W0715 13:40:50.391449 140357147805568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:255: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
W0715 13:40:50.413933 140357147805568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0715 13:40:50.415360 140357147805568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0715 13:40:50.432371 140357147805568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/ba

X_train_head.shape = (36226, 16)
X_train_doc.shape = (36226, 28)
X_train_head.shape = (4124, 16)
X_train_doc.shape = (4124, 28)
{0: 3.5585461689587428, 1: 15.298141891891891, 2: 1.3680513595166164, 3: 0.342154973742869}


...

36311/36311 [==============================] - 15s 406us/step - loss: 0.4208 - acc: 0.8538 - val_loss: 0.3751 - val_acc: 0.8678
 - fnc_score: 0.7644922475206034
 - val_f1_(macro): 0.45561901533792004


Epoch 3/100
36311/36311 [==============================] - 15s 401us/step - loss: 0.3941 - acc: 0.8614 - val_loss: 0.3705 - val_acc: 0.8730
 - fnc_score: 0.7700796200586674
 - val_f1_(macro): 0.4829602157639531


Epoch 4/100
36311/36311 [==============================] - 15s 405us/step - loss: 0.3849 - acc: 0.8648 - val_loss: 0.3743 - val_acc: 0.8690
 - fnc_score: 0.7982958513758905
 - val_f1_(macro): 0.48060443321720836


Epoch 5/100
36311/36311 [==============================] - 15s 406us/step - loss: 0.3731 - acc: 0.8677 - val_loss: 0.3433 - val_acc: 0.8821
 - fnc_score: 0.8016482748987289
 - val_f1_(macro): 0.49151277652986075


Epoch 6/100
36311/36311 [==============================] - 15s 401us/step - loss: 0.3632 - acc: 0.8713 - val_loss: 0.3478 - val_acc: 0.8767
 - fnc_score: 0.7

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:255: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


Starting training 
xtrain shape:(36406, 44)
ytrain shape:(36406,)
Train on 36406 samples, validate on 3944 samples
Epoch 1/100
36406/36406 [==============================] - 21s 564us/step - loss: 0.5319 - acc: 0.8286 - val_loss: 0.3997 - val_acc: 0.8527
 - fnc_score: 0.7330090141651167
 - val_f1_(macro): 0.4053331799438811


Epoch 2/100


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


36406/36406 [==============================] - 15s 400us/step - loss: 0.4232 - acc: 0.8523 - val_loss: 0.3732 - val_acc: 0.8654
 - fnc_score: 0.7574760337673487
 - val_f1_(macro): 0.4570924855758262


Epoch 3/100
36406/36406 [==============================] - 15s 401us/step - loss: 0.3999 - acc: 0.8602 - val_loss: 0.3614 - val_acc: 0.8707
 - fnc_score: 0.7690656746315638
 - val_f1_(macro): 0.45595546495681105


Epoch 4/100
36406/36406 [==============================] - 15s 401us/step - loss: 0.3802 - acc: 0.8669 - val_loss: 0.3535 - val_acc: 0.8674
 - fnc_score: 0.7689225926455859
 - val_f1_(macro): 0.4595311532402958


Epoch 5/100
36406/36406 [==============================] - 15s 399us/step - loss: 0.3693 - acc: 0.8702 - val_loss: 0.3430 - val_acc: 0.8755
 - fnc_score: 0.7880955787666333
 - val_f1_(macro): 0.46324761449848917


Epoch 6/100
36406/36406 [==============================] - 15s 405us/step - loss: 0.3677 - acc: 0.8705 - val_loss: 0.3475 - val_acc: 0.8758
 - fnc_score: 0.78

In [0]:
X_competition_LSTM, X_competition_MLP = split_X(X_competition, MAX_LENGTH)
predicted_one_hot_competition = model.predict(x=[X_competition_LSTM, X_competition_MLP])
predicted = np.argmax(predicted_one_hot, axis=-1)
y_competition_one_hot = convert_data_to_one_hot(np.array(y_competition))
y_competition = np.argmax(y_competition_one_hot, axis=-1)
predicted = [LABELS[int(a)] for a in predicted]


actual = [LABELS[int(a)] for a in y_competition]

print("Scores on the test set")
report_score(actual, predicted)

X_train_head.shape = (25413, 16)
X_train_doc.shape = (25413, 28)
Scores on the test set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    21     |     0     |    189    |    488    |
-------------------------------------------------------------
| disagree  |     5     |     1     |    41     |    199    |
-------------------------------------------------------------
|  discuss  |    51     |     2     |    376    |   1249    |
-------------------------------------------------------------
| unrelated |    229    |     9     |   1534    |   5228    |
-------------------------------------------------------------
Score: 1777.0 out of 11651.25	(15.251582448235167%)


15.251582448235167

In [0]:
predicted


['unrelated',
 'agree',
 'unrelated',
 'unrelated',
 'discuss',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'agree',
 'unrelated',
 'unrelated',
 'unrelated',
 'discuss',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'discuss',
 'unrelated',
 'unrelated',
 'discuss',
 'unrelated',
 'unrelated',
 'discuss',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'agree',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'discuss',
 'unrelated',
 'unrelated',
 'unrelated',
 'discuss',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'discuss',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'discuss',
 'unrelated',
 'unrelated',
 'discuss',
 'discuss',
 'unrelated',
 'unrelated',
 'discuss',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',


In [0]:
actual


['unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'agree',
 'unrelated',
 'discuss',
 'unrelated',
 'discuss',
 'agree',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'discuss',
 'agree',
 'agree',
 'disagree',
 'discuss',
 'unrelated',
 'unrelated',
 'discuss',
 'unrelated',
 'agree',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'discuss',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'discuss',
 'unrelated',
 'discuss',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'discuss',
 'unrelated',
 'discuss',
 'unrelated',
 'unrelated',
 'unrelated',
 'unrelated',
 'discuss',
 'unrelated',
 'unrelated',
 'unrelated',
 'agree',
 'unrelated',
 'unrelated',
 'unrelated',
 'agree',
 'unrelated',
 'agree',
 'unrelated',
 'agree',
 'unrelat